## Challenge Exercise

Modify your solution to the challenge exercise in d_trainandevaluate.ipynb appropriately. Make sure that you implement training and deployment. Increase the size of your dataset by 10x since you are running on the cloud. Does your accuracy improve?

In [14]:
import os
PROJECT = 'qwiklabs-gcp-bf4d36c16278acd2' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-bf4d36c16278acd2' # REPLACE WITH YOUR BUCKET NAME
REGION = 'europe-west1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

In [15]:
# For Python Code
# Model Info
MODEL_NAME = 'Cylinder_volume'
# Model Version
MODEL_VERSION = 'v1'
# Training Directory name
TRAINING_DIR = 'Cylinder_trained'

In [16]:
# For Bash Code
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TRAINING_DIR'] = TRAINING_DIR 
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

In [17]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [18]:
%%bash
# This command will fail if the Cloud Machine Learning Engine API is not enabled using the link above.
echo "Getting the service account email associated with the Cloud Machine Learning Engine API"

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print (response['serviceAccount'])")  # If this command fails, the Cloud Machine Learning Engine API has not been enabled above.

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET   
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET   # error message (if bucket is empty) can be ignored.  
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET    

Getting the service account email associated with the Cloud Machine Learning Engine API
Authorizing the Cloud ML Service account service-679860517096@cloud-ml.google.com.iam.gserviceaccount.com to access files in qwiklabs-gcp-bf4d36c16278acd2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   235    0   235    0     0    244      0 --:--:-- --:--:-- --:--:--   244
No changes to gs://qwiklabs-gcp-bf4d36c16278acd2/
No changes to gs://qwiklabs-gcp-bf4d36c16278acd2/datalab-backups/europe-west1-b/mydatalabvm/content/daily-20181114182409
No changes to gs://qwiklabs-gcp-bf4d36c16278acd2/datalab-backups/europe-west1-b/mydatalabvm/content/hourly-20181114182409
No changes to gs://qwiklabs-gcp-bf4d36c16278acd2/datalab-backups/europe-west1-b/mydatalabvm/content/weekly-20181114182409
No changes to gs://qwiklabs-gcp-bf4d36c16278acd2/


In [20]:
%%bash
find ${MODEL_NAME}

Cylinder_volume
Cylinder_volume/.ipynb_checkpoints
Cylinder_volume/trainer
Cylinder_volume/trainer/__init__.py
Cylinder_volume/trainer/task.py
Cylinder_volume/trainer/model.py


In [21]:
%%bash
cat ${MODEL_NAME}/trainer/model.py

#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import shutil

tf.logging.set_verbosity(tf.logging.INFO)

# List the CSV columns
CSV_COLUMNS = ['radius', 'height', 'volume', 'key']

#Choose which column is your label
LABEL_COLUMN = 'volume'

# Set the default values for eac

In [22]:
%%bash
echo "Working Directory: ${PWD}"
echo "Head of df_train.csv"
head -1 $PWD/df_train.csv
echo "Head of df_valid.csv"
head -1 $PWD/df_valid.csv

Working Directory: /content/datalab/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow
Head of df_train.csv
0,1.1,1.7,6.3
Head of df_valid.csv
0,1.7,0.6,5.4


In [23]:
%%bash
# Clear Cloud Storage bucket and copy the CSV files to Cloud Storage bucket
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/${MODEL_NAME}/smallinput/
gsutil -m cp ${PWD}/*.csv gs://${BUCKET}/${MODEL_NAME}/smallinput/

qwiklabs-gcp-bf4d36c16278acd2


CommandException: 1 files/objects could not be removed.
Copying file:///content/datalab/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/df_valid.csv [Content-Type=text/csv]...
Copying file:///content/datalab/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/taxi-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/taxi-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/df_train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/taxi-valid.csv [Content-Type=text/csv]...
-
Operation completed over 5 objects/19.0 MiB.                                     


In [24]:
%%bash
OUTDIR=gs://${BUCKET}/${MODEL_NAME}/smallinput/${TRAINING_DIR}
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
# Clear the Cloud Storage Bucket used for the training job
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${BUCKET}/${MODEL_NAME}/smallinput/df_train*" \
   --eval_data_paths="gs://${BUCKET}/${MODEL_NAME}/smallinput/df_valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

gs://qwiklabs-gcp-bf4d36c16278acd2/Cylinder_volume/smallinput/Cylinder_trained europe-west1 Cylinder_volume_181114_184024
jobId: Cylinder_volume_181114_184024
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [Cylinder_volume_181114_184024] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe Cylinder_volume_181114_184024

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs Cylinder_volume_181114_184024


In [25]:
%%bash
gsutil ls gs://${BUCKET}/${MODEL_NAME}/smallinput/${TRAINING_DIR}/export/exporter

gs://qwiklabs-gcp-bf4d36c16278acd2/Cylinder_volume/smallinput/Cylinder_trained/export/exporter/
gs://qwiklabs-gcp-bf4d36c16278acd2/Cylinder_volume/smallinput/Cylinder_trained/export/exporter/1542220962/
gs://qwiklabs-gcp-bf4d36c16278acd2/Cylinder_volume/smallinput/Cylinder_trained/export/exporter/1542221345/


In [27]:
%%bash
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION

Created ml engine model [projects/qwiklabs-gcp-bf4d36c16278acd2/models/Cylinder_volume].


In [28]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/${MODEL_NAME}/smallinput/${TRAINING_DIR}/export/exporter | tail -1)

echo "MODEL_LOCATION = ${MODEL_LOCATION}"

gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

MODEL_LOCATION = gs://qwiklabs-gcp-bf4d36c16278acd2/Cylinder_volume/smallinput/Cylinder_trained/export/exporter/1542221345/


Creating version (this might take a few minutes)......
..................................................................................................................................................................................................................................................................................................................................................................................................................done.


In [29]:
%%writefile ./test.json
{"radius": 1.2,"height": 1.8}

Writing ./test.json


In [30]:
%%bash
gcloud ml-engine predict --model=${MODEL_NAME} --version=${MODEL_VERSION} --json-instances=./test.json

PREDICTIONS
[1.1839938163757324]


### Clean-up

#### Delete Model : step 1 - remove version info 
Before an existing cloud model can be removed, it must have any version info removed.  

In [ ]:
%%bash
gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}

#### Delete model: step 2 - remove existing model
Now that the version info is removed from an existing model, the actual model can be removed.  

In [ ]:
%%bash
gcloud ml-engine models delete ${MODEL_NAME}

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License